# Guardrails parser

This output parser invokes [Guardrails](https://docs.guardrailsai.com/) to parse the output of a language model, given a schema defined either in XML or as a pydantic model. Guardrails validates strings, or fields in a JSON, using [its predefined validators](https://docs.guardrailsai.com/api_reference/validators/), or a custom validator you define.

In [10]:
from guardrails import Guard
from guardrails.validators import LowerCase, TwoWords, UpperCase
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import GuardrailsOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field

## With strings

In [11]:
guard = Guard.from_string([LowerCase(on_fail="fix")])

guardrails_parser = GuardrailsOutputParser.from_guard(guard)

prompt = PromptTemplate.from_template(
    "What is a good name for a company that makes {product}?"
)

runnable = prompt | ChatOpenAI() | guardrails_parser

runnable.invoke({"product": "colorful socks"})

'colorful comforts'

## With pydantic

In [20]:
# Here's an example of a Guard constructed from a pydantic model


class Pet(BaseModel):
    pet_type: str = Field(validators=[UpperCase(on_fail="fix")])
    name: str = Field(validators=[TwoWords(on_fail="fix")])


prompt = """
    What kind of pet should I get and what should I name it?

    ${gr.complete_json_suffix_v2}
"""

guard = Guard.from_pydantic(Pet, prompt=prompt)
parser = GuardrailsOutputParser.from_guard(guard)

prompt_template = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{prompt}\n",
    input_variables=["prompt"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

runnable = prompt_template | ChatOpenAI() | parser

runnable.invoke(
    {
        "product": "colorful socks",
        "prompt": prompt,
    }
)

{'pet_type': 'DOG', 'name': 'Buddy'}

### With XML

In [19]:
# Here's another example, of a Guard constructed from an XML RAIL schema

rail_spec = """
<rail version="0.1">
<output>
    <string name="pet_type" validators="uppercase" />
    <string name="name" validators="two-words" />
</output>
<prompt>
    What kind of pet should I get and what should I name it?
    ${gr.complete_json_suffix_v2}
</prompt>
</rail>
"""

guard = Guard.from_rail_string(rail_spec)
parser = GuardrailsOutputParser.from_guard(guard)

prompt_template = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{prompt}\n",
    input_variables=["prompt"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

runnable = prompt_template | ChatOpenAI() | parser

runnable.invoke(
    {
        "product": "colorful socks",
        "prompt": prompt,
    }
)

{'pet_type': 'DOG', 'name': 'Buddy'}